In [1]:
# PART ONE: Data Reading
import pandas as pd
import numpy as np
from scipy.stats import skew
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
def read_data():
    #step1:reading csv data
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    #train.head()   # take a brief look at training data
    all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition'])) # concat training&test data
    return train,test,all_data
#train,test,all_data = read_data()
def model_input():
    alldata_after_filling_missing_skew = pd.read_csv('../input/alldata_after_filling_missing_skew.csv')
    y = pd.read_csv('../input/train_label_skew',header=None)
    test_label = pd.read_csv('../input/test_id',header=None)
    return alldata_after_filling_missing_skew,y,test_label
#alldata_after_filling_missing_skew,y,test_label = model_input()

In [3]:
alldata_after_filling_missing_skew,y,test_label = model_input()

In [8]:
# add new feature 
feat_trial = (alldata_after_filling_missing_skew['1stFlrSF'] + alldata_after_filling_missing_skew['2ndFlrSF']).copy()
print("Skewness of the original intended feature:",skew(feat_trial))
print("Skewness of transformed feature", skew(np.log1p(feat_trial)))
#feat_trial = np.log1p(feat_trial)  # log transform
alldata_after_filling_missing_skew['1stFlr_2ndFlr_Sf'] = feat_trial

('Skewness of the original intended feature:', 0.31460273921961057)
('Skewness of transformed feature', 0.28503200531552075)


In [11]:
feat_trial = (alldata_after_filling_missing_skew['1stFlr_2ndFlr_Sf'] + alldata_after_filling_missing_skew['LowQualFinSF'] + alldata_after_filling_missing_skew['GrLivArea']).copy()
print("Skewness of the original intended feature:",skew(feat_trial))
print("Skewness of transformed feature", skew(np.log1p(feat_trial)))
alldata_after_filling_missing_skew['All_Liv_SF'] = feat_trial

('Skewness of the original intended feature:', 0.38913221266126463)
('Skewness of transformed feature', 0.2942650304316986)


In [14]:
alldata_after_filling_missing_skew.to_csv('../input/alldata_after_filling_missing_skew_newfea.csv',index=False)

In [15]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1)
model_xgb.fit(alldata_after_filling_missing_skew.iloc[:1460],y)
pre_val = np.expm1(pd.DataFrame(model_xgb.predict(alldata_after_filling_missing_skew.iloc[1460:])))
test_label = pd.read_csv('../input/test_id',header=None)
result = pd.DataFrame()
result['Id'] = test_label[0]
result['SalePrice'] = pre_val[0]
result.to_csv('../input/result_xgb_1128_newfea_nofeaselec_skew.csv',index=None)